In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random
import numpy as np
import pandas as pd
import os
# %load_ext autoreload
# %autoreload 2

# Utility Functions

In [2]:
def make_mixed_regression(n_samples, n_features, n_categories):
    X,y = make_regression(n_samples=n_samples, n_features=n_features, random_state=42, n_informative=5, n_targets=2)
    cat_cols = random.choices(list(range(X.shape[-1])),k=n_categories)
    num_cols = [i for i in range(X.shape[-1]) if i not in cat_cols]
    for col in cat_cols:
        X[:,col] = pd.qcut(X[:,col], q=4).codes.astype(int)
    col_names = [] 
    num_col_names=[]
    cat_col_names=[]
    for i in range(X.shape[-1]):
        if i in cat_cols:
            col_names.append(f"cat_col_{i}")
            cat_col_names.append(f"cat_col_{i}")
        if i in num_cols:
            col_names.append(f"num_col_{i}")
            num_col_names.append(f"num_col_{i}")
    X = pd.DataFrame(X, columns=col_names)
    y = pd.DataFrame(y, columns=["target_1","target_2"])
    data = X.join(y)
    return data, cat_col_names, num_col_names

def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = mean_squared_error(y_true, y_pred)
    val_f1 = mean_absolute_error(y_true, y_pred)
    print(f"{tag} MSE: {val_acc} | {tag} MAE: {val_f1}")
    return val_acc, val_f1

# Generate Synthetic Data 

First of all, let's create a synthetic data which is a mix of numerical and categorical features and have multiple targets for regression

In [3]:
data, cat_col_names, num_col_names = make_mixed_regression(n_samples=10000, n_features=20, n_categories=4)
target_cols = ['target_1','target_2']
train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

# Importing the Library

In [4]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GatedAdditiveTreeEnsembleConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [5]:
batch_size = 1024 #Will set the same in the Trainer YAML file
steps_per_epoch = int(train.shape[0]/1024)
epochs = 20

## Basic

**Define the Configs**

In the Basic tutorial, we saw how we declare these params programatically. We can also use YAML files to manage the configuration. In that case, we just need to pass in the path to the file as the argument in `TabularModel`. Let's use a YAML file for TrainerConfig.

For the Learning Rate Scheduler, let's use a OneCycleLR popularized by fast.ai.

In [6]:
results = []

In [7]:
data_config = DataConfig(
    target=target_cols, #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)

optimizer_config = OptimizerConfig(lr_scheduler="OneCycleLR", lr_scheduler_params={"max_lr":0.00478, "epochs": epochs, "steps_per_epoch":steps_per_epoch})

# DEPRECATED
# prediction head is defined separately now and head & head_config will be made
# mandatory in future releases
# model_config = CategoryEmbeddingModelConfig(
#     task="regression",
#     layers="1024-512-512",  # Number of nodes in each layer
#     activation="LeakyReLU", # Activation between each layers
#     learning_rate = 1e-3
# )

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="64-32-16",
    activation="LeakyReLU",
    dropout=0.1,
    initialization="kaiming",
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
    learning_rate = 1e-3
)


**Trainer Config YAML file**
```yaml
batch_size: 1024
fast_dev_run: false
max_epochs: 20
min_epochs: 1
accelerator: 'auto'
devices: -1
accumulate_grad_batches: 1
auto_lr_find: true
check_val_every_n_epoch: 1
gradient_clip_val: 0.0
overfit_batches: 0.0
profiler: null
early_stopping: null
early_stopping_min_delta: 0.001
early_stopping_mode: min
early_stopping_patience: 3
checkpoints: valid_loss
checkpoints_path: saved_models
checkpoints_mode: min
checkpoints_save_top_k: 1
load_best: true
track_grad_norm: -1

```

In [8]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config="../examples/yaml_config/trainer_config.yml",
)

#### High-level API

In [9]:
# tabular_model.fit(train=train, validation=val)

#### Low-Level API

Enables you to reuse the datamodule to train multiple models with different configs. We can also implement more complex validation schemes like K-Fold Cross Validation efficiently.

In [10]:
datamodule = tabular_model.prepare_dataloader(
                train=train, validation=val, seed=42
            )
model = tabular_model.prepare_model(
            datamodule
        )
tabular_model.train(model, datamodule)

/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/l

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.10964781961431852
Restoring states from the checkpoint path at /home/manujosephv/pytorch_tabular/docs/.lr_find_84ecc6c9-bf82-47e0-b215-ce6774acc9db.ckpt
Restored all states from the checkpoint file at /home/manujosephv/pytorch_tabular/docs/.lr_find_84ecc6c9-bf82-47e0-b215-ce6774acc9db.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                      | Params
---------------------------------------------------------------
0 | _backbone        | CategoryEmbeddingBackbone | 4.5 K 
1 | _embedding_layer | Embedding1dLayer          | 92    
2 | head             | LinearHead                | 34    
3 | loss             | MSELoss                   | 0     
---------------------------------------------------------------
4.6 K     Trainable params
0         Non-trainable params
4.6 K     Total params
0.018     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_lightning/utilities/cloud_io.py:41: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.load` has been deprecated in v1.8.0 and will be removed in v1.10.0. This function is internal but you can copy over its implementation.
  rank_zero_deprecation(


In [11]:
result = tabular_model.evaluate(test)

/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/l

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     3860.49951171875      │
│        test_loss_0        │     1658.809814453125     │
│        test_loss_1        │     2201.68994140625      │
│  test_mean_squared_error  │     3860.49951171875      │
│ test_mean_squared_error_0 │     1658.809814453125     │
│ test_mean_squared_error_1 │     2201.68994140625      │
└───────────────────────────┴───────────────────────────┘

We can see the metrics and loss for each target and a total loss/metric. We can pin the EarlyStopping or the Checkpoint Saving on any one of these metrics

In [12]:
pred_df = tabular_model.predict(test)
pred_df.head()

/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/l

Generating Predictions...:   0%|          | 0/3 [00:00<?, ?it/s]

,num_col_0,num_col_1,num_col_2,cat_col_3,num_col_4,num_col_5,num_col_6,num_col_7,num_col_8,num_col_9,...,cat_col_14,num_col_15,num_col_16,num_col_17,num_col_18,cat_col_19,target_1,target_2,target_1_prediction,target_2_prediction
6252,0.087964,2.258349,0.441456,0.0,-0.182298,0.381874,0.457820,-1.089108,-0.608747,0.659034,...,3.0,-0.209956,-0.025792,-0.295642,-1.723547,3.0,1.506144,52.811114,10.054810,26.786684
4684,1.032769,0.416355,1.171905,2.0,-2.029677,-0.660151,0.592990,0.129360,0.295198,-0.692314,...,2.0,0.425943,0.001516,-0.258499,-1.083438,2.0,119.189386,93.043074,119.440979,122.915184
1731,-0.652624,-1.583903,-2.423879,0.0,-0.452306,-1.430775,-0.676392,0.196521,1.440117,0.760415,...,3.0,0.840644,0.709004,-0.681052,0.128104,1.0,174.493212,239.079479,155.372498,188.779907
4742,-0.451170,0.239125,0.574557,0.0,-0.875318,0.956782,-0.732752,0.853738,0.713685,-0.373434,...,1.0,0.275938,-0.720602,-0.758199,0.161861,2.0,-16.423023,-114.608320,-49.305008,-106.651672
4521,0.010387,0.323700,-2.604310,1.0,-0.979168,1.944288,0.448619,-2.838460,0.532355,-2.779626,...,1.0,-0.846792,0.109045,-0.299561,0.051376,1.0,11.986265,-16.142641,18.009151,22.230291


In [13]:
print("Target 1")
val_mse_1, val_mae_1 = print_metrics(test['target_1'], pred_df["target_1_prediction"], tag="Holdout")
print("Target 2")
val_mse_2, val_mae_2 = print_metrics(test['target_2'], pred_df["target_2_prediction"], tag="Holdout")

Target 1
Holdout MSE: 1658.8098629750177 | Holdout MAE: 27.35465466497158
Target 2
Holdout MSE: 2201.6900239822185 | Holdout MAE: 30.658695150971326


In [14]:
results.append({
    "Mode": "Basic (CategoryEmbedding)",
    "Target 1: MSE": val_mse_1,
    "Target 1: MAE": val_mae_1,
    "Target 2: MSE": val_mse_2,
    "Target 2: MAE": val_mae_2,
})

## Advanced

Let's do the following:
1. A data transform for the continuous columns
2. Set Target Ranges for the multiple targets
3. Use NODE model
4. A Custom Optimizer

In [15]:
#Since we are using a lower learning rate, increasing the epochs
batch_size = 512
steps_per_epoch = int(train.shape[0]/batch_size)
epochs = 50

In [16]:
data_config = DataConfig(
    target=target_cols, #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal"
)


trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=batch_size,
    max_epochs=epochs,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    load_best=True, # After training, load the best checkpoint
    accelerator="auto", # can be 'cpu','gpu', 'tpu', or 'ipu' 
    devices=-1 # -1 means use all available
)


optimizer_config = OptimizerConfig(
    lr_scheduler="OneCycleLR",
    lr_scheduler_params={
        "max_lr":2e-3, 
        "epochs": epochs, 
        "steps_per_epoch":steps_per_epoch
    }
)

head_config = LinearHeadConfig(
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

model_config = GatedAdditiveTreeEnsembleConfig(
    task="regression",
    num_trees=50,
    chain_trees=False,
    share_head_weights=False,
    head = "LinearHead",
    head_config=head_config,
    learning_rate = 1e-3,
    # Setting target_range to restrict output between min and max
    target_range=[(float(train[col].min()),float(train[col].max())) for col in target_cols]
)

In [17]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [19]:
from torch_optimizer import QHAdam
from sklearn.preprocessing import PowerTransformer

In [20]:
tabular_model.fit(train=train, 
                  validation=val, 
                #   target_transform=PowerTransformer(method="yeo-johnson"), 
                  optimizer=QHAdam, # Using a custom optimizer
                  optimizer_params={"nus": (0.7, 1.0), "betas": (0.95, 0.998)})

/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
Global seed set to 42
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a futur

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.0005248074602497723
Restoring states from the checkpoint path at /home/manujosephv/pytorch_tabular/docs/.lr_find_7aeb1237-c1b2-4c4d-b857-5b01bf6b36c1.ckpt
Restored all states from the checkpoint file at /home/manujosephv/pytorch_tabular/docs/.lr_find_7aeb1237-c1b2-4c4d-b857-5b01bf6b36c1.ckpt
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /home/manujosephv/pytorch_tabular/docs/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                       | Params
----------------------------------------------------------------
0 | _backbone        | GatedAdditiveTreesBackbone | 330 K 
1 | _embedding_layer | Embedding1dLayer           | 92    
2 | _head            | CustomHe

Sanity Checking: 0it [00:00, ?it/s]

/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_lightning/utilities/cloud_io.py:41: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.load` has been deprecated in v1.8.0 and will be removed in v1.10.0. This function is internal but you can copy over its implementation.
  rank_zero_deprecation(


In [21]:
result = tabular_model.evaluate(test)

/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/l

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │      736.08349609375      │
│        test_loss_0        │    238.44142150878906     │
│        test_loss_1        │     497.6420593261719     │
│  test_mean_squared_error  │      736.08349609375      │
│ test_mean_squared_error_0 │    238.44142150878906     │
│ test_mean_squared_error_1 │     497.6420593261719     │
└───────────────────────────┴───────────────────────────┘

In [22]:
pred_df = tabular_model.predict(test)
pred_df.head()

/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/lib/python3.10/site-packages/pytorch_tabular/categorical_encoders.py:48: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_encoded.loc[:, col] = X_encoded[col].fillna(NAN_CATEGORY).map(mapping["value"])
/home/manujosephv/pytorch_tabular/.env/tabular_env/l

Generating Predictions...:   0%|          | 0/5 [00:00<?, ?it/s]

,num_col_0,num_col_1,num_col_2,cat_col_3,num_col_4,num_col_5,num_col_6,num_col_7,num_col_8,num_col_9,...,cat_col_14,num_col_15,num_col_16,num_col_17,num_col_18,cat_col_19,target_1,target_2,target_1_prediction,target_2_prediction
6252,0.087964,2.258349,0.441456,0.0,-0.182298,0.381874,0.457820,-1.089108,-0.608747,0.659034,...,3.0,-0.209956,-0.025792,-0.295642,-1.723547,3.0,1.506144,52.811114,5.752899,75.185913
4684,1.032769,0.416355,1.171905,2.0,-2.029677,-0.660151,0.592990,0.129360,0.295198,-0.692314,...,2.0,0.425943,0.001516,-0.258499,-1.083438,2.0,119.189386,93.043074,128.403778,114.819153
1731,-0.652624,-1.583903,-2.423879,0.0,-0.452306,-1.430775,-0.676392,0.196521,1.440117,0.760415,...,3.0,0.840644,0.709004,-0.681052,0.128104,1.0,174.493212,239.079479,172.001495,240.784912
4742,-0.451170,0.239125,0.574557,0.0,-0.875318,0.956782,-0.732752,0.853738,0.713685,-0.373434,...,1.0,0.275938,-0.720602,-0.758199,0.161861,2.0,-16.423023,-114.608320,-20.892853,-129.873932
4521,0.010387,0.323700,-2.604310,1.0,-0.979168,1.944288,0.448619,-2.838460,0.532355,-2.779626,...,1.0,-0.846792,0.109045,-0.299561,0.051376,1.0,11.986265,-16.142641,5.828857,-18.631592


In [23]:
# print("Target 1")
# print_metrics(test['target_1'], pred_df["target_1_prediction"], tag="Holdout")
# print("Target 2")
# print_metrics(test['target_2'], pred_df["target_2_prediction"], tag="Holdout")

In [24]:
print("Target 1")
val_mse_1, val_mae_1 = print_metrics(test['target_1'], pred_df["target_1_prediction"], tag="Holdout")
print("Target 2")
val_mse_2, val_mae_2 = print_metrics(test['target_2'], pred_df["target_2_prediction"], tag="Holdout")

Target 1
Holdout MSE: 238.4415303869453 | Holdout MAE: 9.192527249703767
Target 2
Holdout MSE: 497.6418605542844 | Holdout MAE: 13.692622598845643


In [25]:
results.append({
    "Mode": "Advanced (GATE)",
    "Target 1: MSE": val_mse_1,
    "Target 1: MAE": val_mae_1,
    "Target 2: MSE": val_mse_2,
    "Target 2: MAE": val_mae_2,
})

## Comparison

In [26]:
res_df = pd.DataFrame(results).T
res_df.columns = res_df.iloc[0]
res_df = res_df.iloc[1:].astype(float)
res_df.style.highlight_min(color="lightgreen",axis=1)

Mode,Basic (CategoryEmbedding),Advanced (GATE)
Target 1: MSE,1658.809863,238.441530
Target 1: MAE,27.354655,9.192527
Target 2: MSE,2201.690024,497.641861
Target 2: MAE,30.658695,13.692623
